In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - get_apo_dr16_gaia_dr3.ipynb
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''get Gaia DR3 and APOGEE DR16 data'''

__author__ = "James Lane"

In [ ]:
### Imports
import os,sys
import numpy as np
import dill as pickle

# Set APOGEE version for the package, l33 is for DR16
apogee_results_vers = 'l33'
# Year 7 is appropriate for DR16 (l33)
apo_combined_select_year = 7
os.environ['RESULTS_VERS'] = apogee_results_vers

# APOGEE, Gaia
import apogee.tools.read as apread
import apogee.select as apsel
import apogee.tools as apotools
from gaia_tools import xmatch

# Project-specific
sys.path.append('../../src/')
from ges_mass import util as putil

In [ ]:
### Notebook setup
%load_ext autoreload
%autoreload 2

In [ ]:
### Keywords

# Force load different data products even they exist (overwrite them)?
force_load_allstar = True
force_load_apogeeSF = True
force_load_statIndx = True
force_load_gaia = True

# Keywords for APOGEE allstar (see apread.allStar.__doc__)
allstar_main = True # Only select stars in the main survey
allstar_rmdups = True # Remove duplicates
allstar_rmcomissioning = True # Remove comissioning data
allstar_use_astroNN_abundances = False # replace ASPCAP w/ astroNN abundances
allstar_use_astroNN_distances = True # Add in astroNN distances
allstar_use_astroNN_ages = False # Add in astroNN ages
swap_in_edr3_distances = True # Add in distances calculated with eDR3

# APOGEE and Gaia data release information
apogee_dr = 'dr16' # dr16 or dr17
gaia_dr = 'edr3' # dr2 or edr3

# Sanity
assert apotools.path._APOGEE_REDUX == apogee_results_vers,\
    'Not using correct results version!'

In [ ]:
### Pathing

# Outputs
data_dir_base = '/geir_data/scr/lane/projects/ges-mass/data/gaia_apogee/'
data_dir = data_dir_base+'apogee_'+apogee_dr+'_'+apogee_results_vers+\
                         '_gaia_'+gaia_dr+'/'
os.makedirs(data_dir, exist_ok=True)

allstar_filename = data_dir+'apogee_allstar.npy'
apogeeSF_filename = data_dir+'apogee_SF.dat'
statIndx_filename = data_dir+'apogee_statIndx.npy'
gaia_data_filename = data_dir+'gaia_data.npy'
gaia_apogee_matches_filename = data_dir+'gaia_apogee_matches.npy'

# Get APOGEE Data

In [ ]:
# Get data
if not force_load_allstar and os.path.exists(allstar_filename):
    print('Loading pre-computed allstar from '+allstar_filename)
    allstar = np.load(allstar_filename)
else:
    print('Loading APOGEE from results version: '+apogee_results_vers)
    allstar = apread.allStar(main=allstar_main, rmdups=allstar_rmdups, 
        use_astroNN_abundances=allstar_use_astroNN_abundances,
        use_astroNN_distances=allstar_use_astroNN_distances,
        use_astroNN_ages=allstar_use_astroNN_ages)
    print('Saving allstar as: '+allstar_filename)
    np.save(allstar_filename, allstar)
    # Automatically force the rest because you loaded a new allstar
    force_load_apogeeSF = True
    force_load_statIndx = True
    force_load_gaia = True

# Get the APOGEE statistical sample
This part will take awhile, a few hours probably

In [ ]:
# Get selection function
if not force_load_apogeeSF and os.path.exists(apogeeSF_filename):
    print('Loading pre-computed selection function from '+apogeeSF_filename)
    with open(apogeeSF_filename, 'rb') as f:
        apo = pickle.load(f)
else:
    apo = apsel.apogeeCombinedSelect(year=apo_combined_select_year)
    with open(apogeeSF_filename, 'wb') as f:
        pickle.dump(apo,f)
    # Automatically force the rest
    force_load_statIndx = True
    force_load_gaia = True

# Make statistical sample
if not force_load_statIndx and os.path.exists(statIndx_filename):
    print('Loading pre-computed statistical sample from '+statIndx_filename)
    statIndx = np.load(statIndx_filename)
else:
    statIndx = apo.determine_statistical(allstar)
    np.save(statIndx_filename, statIndx)
    # Automatically force the rest
    force_load_gaia = True

In [ ]:
# Save some information about the APOGEE selection function fields for use 
# elsewhere
apo_field = apo._apogeeField
glons = apo_field['GLON']
glats = apo_field['GLAT']
locids = apo_field['LOCATION_ID']

np.save(data_dir+'apogee_field_glons.npy',glons)
np.save(data_dir+'apogee_field_glats.npy',glats)
np.save(data_dir+'apogee_field_location_ids.npy',locids)
np.save(data_dir+'apogee_fields.npy',apo_field)

## Get the Gaia eDR3 data

In [ ]:
if not force_load_gaia and os.path.exists(gaia_data_filename)\
                       and os.path.exists(gaia_apogee_matches_filename):
    print('Loading Gaia data from '+gaia_data_filename)
    print('Loading Gaia-APOGEE matches from '+gaia_apogee_matches_filename)
    gaia_data = np.load(gaia_data_filename)
    matches_index = np.load(gaia_apogee_matches_filename)
else:
    if gaia_dr == 'edr3':
        xcat_path = 'vizier:I/350/gaiaedr3'
    gaia_data, matches_indx = xmatch.cds(allstar[statIndx], 
                                             colRA='RA', colDec='DEC', 
                                             xcat=xcat_path, 
                                             gaia_all_columns=True)
    np.save(gaia_data_filename, gaia_data)
    np.save(gaia_apogee_matches_filename, matches_indx)

## Swap in Gaia eDR3 AstroNN distances if requested

In [ ]:
if swap_in_edr3_distances:
    allstar_new,_ = putil.swap_in_edr3_distances_to_dr16(allstar,
        keep_old_distances=True,return_match=True)
    
    print('Saving allstar with eDR3 AstroNN distances as: '+allstar_filename)
    np.save(allstar_filename, allstar_new)